In [1]:
import pandas as pd
import numpy as np
import random

import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.51-b03, mixed mode)
  Starting server from C:\Users\Acer\anaconda3\envs\engine\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Acer\AppData\Local\Temp\tmp6gaujgx5
  JVM stdout: C:\Users\Acer\AppData\Local\Temp\tmp6gaujgx5\h2o_Acer_started_from_python.out
  JVM stderr: C:\Users\Acer\AppData\Local\Temp\tmp6gaujgx5\h2o_Acer_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Asia/Bangkok
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,2 months and 8 days
H2O_cluster_name:,H2O_from_python_Acer_hiv4nr
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.501 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [2]:
tasks = pd.read_csv('data/30_2018/task_data.csv')
tasks_df = pd.read_csv('data/30_2018/task_train.csv')

target = 'TaskDelay'

X = tasks_df.copy()
y = X.pop(target)

In [5]:
project_ids = tasks['ProjectID'].unique().tolist()
indices = random.sample(project_ids, int(0.2 * len(project_ids)))
val_indices = random.sample(indices, int(0.5 * len(indices)))
test_indices = [x for x in indices if x not in val_indices]
val_idx = tasks.loc[tasks['ProjectID'].isin(val_indices)].index.tolist()
test_idx = tasks.loc[tasks['ProjectID'].isin(test_indices)].index.tolist()

val_frame = tasks_df[tasks_df.index.isin(val_idx)]
test_frame = tasks_df[tasks_df.index.isin(test_idx)]
train_frame = tasks_df[(~tasks_df.index.isin(test_idx)) & (~tasks_df.index.isin(val_idx))]

In [6]:
train_h2o = h2o.H2OFrame(train_frame)
val_h2o = h2o.H2OFrame(val_frame)
test_h2o = h2o.H2OFrame(test_frame)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [7]:
x = train_h2o.columns
y = 'TaskDelay'  # replace with your actual response column
x.remove(y)

In [ ]:
aml = H2OAutoML(max_runtime_secs=1800,
                seed=42,
                stopping_metric='deviance', 
                nfolds=0)
aml.train(x=x, y=y, training_frame=train_h2o, validation_frame=val_h2o)

In [ ]:
lb = aml.leaderboard
print(lb)

In [ ]:
best = aml.leader
best.model_performance(test_h2o)

In [ ]:
h2o.save_model(model=best, path='model', force=True)

In [3]:
loaded_model = h2o.load_model("model/30_2018_30")

In [8]:
loaded_model.model_performance(test_h2o)

ModelMetricsRegression: deeplearning
** Reported on test data. **

MSE: 1.997474795582276
RMSE: 1.413320485800116
MAE: 1.0530480904799355
RMSLE: 0.48725454725594153
Mean Residual Deviance: 1.997474795582276

In [ ]:
preds = loaded_model.predict(test_h2o)
test_h2o['Preds'] = preds 
test_h2o.head(50)

In [ ]:
preds.as_data_frame().iat[0, 0]